In [ ]:
# 1- Leggo un wfset full-streaming
# 2- Sottraggo la baseline e inverto il segno
# 3- Certo i picchi con MyAnaPeak_NEW
# 4- Seleziono solo le wf con un picco associato al beam (tra due tick, tramite lo slice)
# 5- Taglio la finestra intorno al picco indentificando 1/3 dell'ampiezza del picco
# 6- Convoluzione + fit per estrapolare il numero di PE
# 7- Plot del light yield

# NOTE:
# Lo slice adcs funziona per selezionare gli adcs, 
# Tutte le info della wf non vengono modificare (timestamp, daq_window_timestamp, run_number,record_number,endpoint,channel,trigger_type), ad eccezione dello starting_tick che di default è 0 di solito
# Non cambia nessun attributo per il wfset, compreso points_per_wf 187520 (attenzione)


In [ ]:
import os 
import importlib
import math

from waffles.input_output.hdf5_structured import load_structured_waveformset
from waffles.data_classes.WaveformSet import WaveformSet
from waffles.data_classes.Waveform import Waveform
from waffles.np04_data.ProtoDUNE_HD_APA_maps import APA_map # apa map
from waffles.data_classes.ChannelWsGrid import ChannelWsGrid # channel grid


import waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots as utils_plots_module
from waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots import *


import waffles.np04_analysis.lightyield_vs_energy.scripts.MyAnaPeak_NEW as MyAnaPeak_NEW_module

import waffles.np04_analysis.lightyield_vs_energy.scripts.Renan_scripts.function as renan_function_module
from waffles.np04_analysis.lightyield_vs_energy.scripts.Renan_scripts.function import *


import waffles.np04_analysis.lightyield_vs_energy.scripts.Renan_scripts.photon_fix_v2  as renan_photon_fix_module
from waffles.np04_analysis.lightyield_vs_energy.scripts.Renan_scripts.photon_fix_v2 import photon_counter

import waffles.np04_analysis.lightyield_vs_energy.scripts.MyAnaConvolution as MyAnaConvolution_module
from waffles.np04_analysis.lightyield_vs_energy.scripts.MyAnaConvolution import MyAnaConvolution


In [ ]:
# Reading full streaming data

run_folder = "/afs/cern.ch/work/a/anbalbon/public/reading_rucio/run027361/full_streaming/no_selection"
fname = "processed_np04hd_raw_run027361_0001_dataflow0_datawriter_0_20240622T004415.hdf5_structured.hdf5"
filepath = os.path.join(run_folder, fname)
prova_wfset = load_structured_waveformset(filepath)  # load HDF5 structured waveform set  

In [ ]:
importlib.reload(utils_plots_module)
from waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots import *
plotting_overlap_wf_PEAK_NEW(prova_wfset, n_wf= 2, show  = True, peak_bool = False, peak_beam_bool = False, analysis_label = '')

In [ ]:
# Function to remove baseline and reverse the signal (if needed --> good for find_peaks)

from waffles.data_classes.WaveformAdcs import WaveformAdcs
from waffles.Exceptions import GenerateExceptionMessage

def my_subtract_baseline(
        waveform: WaveformAdcs,
        baseline_analysis_label: str,
        inversion: bool
) -> None:
    """This method overwrites the adcs method of the given
    WaveformAdcs object, by subtracting its baseline.

    waveform: WaveformAdcs
        The waveform whose adcs will be modified
    baseline_analysis_label: str
        The baseline to subtract must be available 
        under the 'baseline' key of the result of the analysis
        whose label is given by this parameter, i.e. in
        waveform.analyses[analysis_label].result['baseline']
    inversion: bool
        If True, the baseline-subtracted signal is inverted.
        If False, the baseline-subtracted signal is not inverted.
    """

    try:
        baseline = waveform.analyses[baseline_analysis_label].result['baseline']

    except KeyError:
        raise Exception(
            GenerateExceptionMessage(
                1,
                "subtract_baseline()",
                f"The given waveform does not have the analysis"
                f" '{baseline_analysis_label}' in its analyses "
                "attribute, or it does, but the 'baseline' key "
                "is not present in its result."
            )
        )
    
    if inversion:
        waveform._WaveformAdcs__set_adcs(
            baseline - waveform.adcs
        )
    else:
        waveform._WaveformAdcs__set_adcs(
            waveform.adcs - baseline
        )

    return

In [ ]:
#Tolgo la baseline subito e inverto 

from waffles.utils.baseline.baseline_utils import subtract_baseline # baseline subtraction
from waffles.utils.baseline.WindowBaseliner import WindowBaseliner
from waffles.data_classes.StoreWfAna import StoreWfAna 


baseline_limits = [0, 200]
baseliner_std_cut = 3.
baseliner_type = 'mean'
baseline_analysis_label = 'baseliner'
null_baseline_analysis_label = 'null_baseliner'


# Baseline
baseliner_input_parameters = IPDict({'baseline_limits': baseline_limits, 'std_cut': baseliner_std_cut, 'type': baseliner_type})    
checks_kwargs = IPDict({'points_no': prova_wfset.points_per_wf})
_ = prova_wfset.analyse(baseline_analysis_label, WindowBaseliner, baseliner_input_parameters, checks_kwargs=checks_kwargs, overwrite=True)


# Remove baseline
prova_wfset.apply(my_subtract_baseline, baseline_analysis_label, inversion = True, show_progress=False)

In [ ]:
importlib.reload(utils_plots_module)
from waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots import *
plotting_overlap_wf_PEAK_NEW(prova_wfset, n_wf= 5, show  = True, peak_bool = False, peak_beam_bool = False, analysis_label = '')

In [ ]:
# Goal: search for peak 
importlib.reload(MyAnaPeak_NEW_module)
from waffles.np04_analysis.lightyield_vs_energy.scripts.MyAnaPeak_NEW import MyAnaPeak_NEW
analysis_class = MyAnaPeak_NEW
analysis_label = 'finding_peaks'

ip = IPDict(baseline_limits= [0, 200],
            n_std = 5,
            peak_distance= 500,
            beam_timeoffset_limits = [-40, -20],
            signal_sign = "positive"
            )

_ = prova_wfset.analyse(label=analysis_label ,analysis_class=analysis_class, input_parameters=ip, checks_kwargs = IPDict({'points_no': prova_wfset.points_per_wf}), overwrite=True)


In [ ]:
importlib.reload(utils_plots_module)
from waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots import *
plotting_overlap_wf_PEAK_NEW(prova_wfset, n_wf= 2, show  = True, peak_bool = True, peak_beam_bool = True, analysis_label = analysis_label)

In [ ]:
# Selecting beam waveforms with just one peak 
def filter_beam_peaks_0(waveform: Waveform, analysis_label) -> bool:
    if len(waveform.analyses[analysis_label].result['beam_peak_index']) == 1:
        return True
    else:
        return False
    
prova_wfset = WaveformSet.from_filtered_WaveformSet(prova_wfset, filter_beam_peaks_0, analysis_label)


In [ ]:
# To cut full-streaming waveforms around the peak

# The waveform must have the beaseline around zero and the signal inverted (positive peaks)


from waffles.data_classes.WaveformAdcs import WaveformAdcs
from waffles.Exceptions import GenerateExceptionMessage
import numpy as np

def cut_full_streaming_window_and_inverting(
        waveform: WaveformAdcs,
        analysis_label: str,
        amplitude_threshold: float = 0.3,
        pre_peak_window: int = 50,
        window_timeticks_length: int = 588 #check the template, to have the same length
) -> None:
    """This method overwrites the adcs method of the given
    WaveformAdcs object.

    waveform: WaveformAdcs
        The waveform whose adcs will be modified
    analysis_label: str
        Name of the analysis including the peak information
    amplitude_threshold: float
        Percentage of the peak amplitude to define the starting point for the cut 
    """

    try:
        peak_index = waveform.analyses[analysis_label].result['beam_peak_index']
        baseline = waveform.analyses[analysis_label].result['mean_baseline']
    except KeyError:
        raise Exception(
            GenerateExceptionMessage(
                1,
                "Problem finding 'beam_peak_index' or 'beam_peak_amplitude' "
            )
        )
    
    # if len(peak_index) != 1:
    #     print("⚠️ peak_index length != 1, skipping…")
    #     return
    # else:
    peak_index = waveform.analyses[analysis_label].result['beam_peak_index'][0]
    peak_amplitude = waveform.analyses[analysis_label].result['beam_peak_amplitude'][0]

    # Searching for the index associated to peak_amplitude * amplitude_threshold (% of the raising edge)
    amplitude_threshold_dac = peak_amplitude * amplitude_threshold

    window = waveform.adcs[peak_index-pre_peak_window : peak_index]
    idx_in_window = np.abs(window - amplitude_threshold_dac).argmin()
    idx = (peak_index - pre_peak_window) + idx_in_window

    start = idx - pre_peak_window
    stop  = start + window_timeticks_length

    if start < 0:
        start = 0
        stop = window_timeticks_length

    if stop > len(waveform.adcs):
        stop = len(waveform.adcs)
        start = stop - window_timeticks_length

    # Ultimo controllo: la lunghezza DEVE essere esattamente quella
    if stop - start != window_timeticks_length:
        print("Skipping: cannot enforce fixed window length")

    # ---- PARTE 5: slicing garantito omogeneo
    waveform._Waveform__slice_adcs(start, stop)

    return 


In [ ]:
prova_wfset.apply(cut_full_streaming_window_and_inverting, analysis_label = analysis_label ,show_progress=False)

In [ ]:
importlib.reload(utils_plots_module)
from waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots import *
plotting_overlap_wf_PEAK_NEW(prova_wfset, n_wf= 3, show  = True, peak_bool = False, peak_beam_bool = True, analysis_label = analysis_label)

In [ ]:
# Reading templates and createing a datagrame with three column: 'endpoint' 'channel' 'Template_avg' (i.e. adcs)
directory_path = '/afs/cern.ch/work/a/anbalbon/private/waffles/src/waffles/np04_analysis/lightyield_vs_energy/scripts/Renan_scripts/v_27xxx_fix_avg'
template_files = os.listdir(directory_path)

# Filter only files (not directories)
template_files = [f for f in template_files if os.path.isfile(os.path.join(directory_path, f))]

edp_list = []
ch_list  = []
adc_list = []

for f in template_files:
    try:
        parts = f.split()
        if len(parts) < 3 or "_" not in parts[2]:
            print(f"Unexpected filename format: {f}")
            continue

        edp = parts[2].split("_")[0]
        ch = parts[2].split("_")[1].split('.')[0]

        with open(os.path.join(directory_path, f), 'rb') as file:
            temp = pickle.load(file)

        adc_list.append(temp)
        edp_list.append(edp)
        ch_list.append(ch)

    except Exception as e:
        print(f"Error loading {f}: {e}")

df_template = pd.DataFrame({
    'endpoint' : edp_list,
    'channel' : ch_list,
    'Template_avg': adc_list})
df_template["endpoint"] = df_template["endpoint"].astype(int)
df_template["channel"]  = df_template["channel"].astype(int)



In [ ]:
importlib.reload(renan_function_module) 
from waffles.np04_analysis.lightyield_vs_energy.scripts.Renan_scripts.function import *

importlib.reload(renan_photon_fix_module) 
from waffles.np04_analysis.lightyield_vs_energy.scripts.Renan_scripts.photon_fix_v2 import photon_counter

apa = 1
end = 105
ch = 7

grid_apa_FS = ChannelWsGrid(APA_map[apa], prova_wfset, compute_calib_histo=False)

photon_counter(grid_apa_FS.ch_wf_sets[end][ch].waveforms[0].adcs, df_template.loc[(df_template['endpoint'] == end) & (df_template['channel'] == ch), 'Template_avg'].iloc[0], end, ch, 256, debug = False)

In [ ]:
# print("\nFS_wfset waveformset:")
# print("points_per_wf:", FS_wfset.points_per_wf)
# print("runs:", FS_wfset.runs)
# print("record_numbers:", FS_wfset.record_numbers)
# print("available_channels:", FS_wfset.available_channels)
# print("mean_adcs:", FS_wfset.mean_adcs)
# print("mean_adcs_idcs:", FS_wfset.mean_adcs_idcs)

# print("\nprova_wfset waveformset:")
# print("points_per_wf:", prova_wfset.points_per_wf)
# print("runs:", prova_wfset.runs)
# print("record_numbers:", prova_wfset.record_numbers)
# print("available_channels:", prova_wfset.available_channels)
# print("mean_adcs:", prova_wfset.mean_adcs)
# print("mean_adcs_idcs:", prova_wfset.mean_adcs_idcs)

# print("\n --------------------------------------------- \n")
# i=2
# print("\nprova_wfset waveform:")
# print("timestamp:", prova_wfset.waveforms[i].timestamp)
# print("daq_window_timestamp:", prova_wfset.waveforms[i].daq_window_timestamp)
# print("run_number:", prova_wfset.waveforms[i].run_number)
# print("record_number:", prova_wfset.waveforms[i].record_number)
# print("endpoint:", prova_wfset.waveforms[i].endpoint)
# print("channel:", prova_wfset.waveforms[i].channel)
# print("starting_tick:", prova_wfset.waveforms[i].starting_tick)
# print("trigger_type:", prova_wfset.waveforms[i].trigger_type)

# print("\n\FS_wfset waveform:")
# print("timestamp:", FS_wfset.waveforms[i].timestamp)
# print("daq_window_timestamp:", FS_wfset.waveforms[i].daq_window_timestamp)
# print("run_number:", FS_wfset.waveforms[i].run_number)
# print("record_number:", FS_wfset.waveforms[i].record_number)
# print("endpoint:", FS_wfset.waveforms[i].endpoint)
# print("channel:", FS_wfset.waveforms[i].channel)
# print("starting_tick:", FS_wfset.waveforms[i].starting_tick)
# print("trigger_type:", FS_wfset.waveforms[i].trigger_type)

In [ ]:
# Implementing evaluation of #pe as WfAna analysis  
importlib.reload(MyAnaConvolution_module)
from waffles.np04_analysis.lightyield_vs_energy.scripts.MyAnaConvolution import MyAnaConvolution
analysis_class = MyAnaConvolution
analysis_label = 'searching_pe'

_ = prova_wfset.analyse(label=analysis_label ,analysis_class=analysis_class, input_parameters=IPDict(df_template= df_template), checks_kwargs = IPDict({'points_no': prova_wfset.points_per_wf}), overwrite=True)


In [ ]:
apa = 1
end = 107
ch = 10

grid_apa_FS = ChannelWsGrid(APA_map[apa], prova_wfset, compute_calib_histo=False)

print(grid_apa_FS.ch_wf_sets[end][ch].waveforms[20].analyses[analysis_label].result['n_pe'])


In [ ]:
list_pe = []


for run, run_dic in prova_wfset.available_channels.items():
    for endpoint, channel_list in run_dic.items():
        for channel in channel_list:
            for i in range(len(grid_apa_FS.ch_wf_sets[endpoint][channel].waveforms)):
                list_pe.append(grid_apa_FS.ch_wf_sets[endpoint][channel].waveforms[i].analyses[analysis_label].result['n_pe'])

print(len(list_pe))
list_pe = [x for x in list_pe if not math.isnan(x)]
print(len(list_pe))

In [ ]:
plt.figure()
plt.hist(list_pe, bins= 20, label=f"Mean = {np.mean(list_pe):.2f}\nσ = {np.std(list_pe):.2f}")
plt.xlabel("N° photoelectrons")
plt.ylabel("Counts")
plt.title("Photoelectron distribution")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText

def check_convolution_result(wf, analysis_label):
    try:
        template = df_template.loc[(df_template['endpoint'] == wf.endpoint) & (df_template['channel'] == wf.channel), 'Template_avg'].iloc[0]
    except Exception as e:
        print(f"Error retrieving template for endpoint {wf.endpoint} and channel {wf.channel}: {e}")
        return
    wf_adcs = wf.adcs

    plt.rcParams.update({
    'font.size': 12,
    'axes.titlesize': 14,
    'axes.labelsize': 14,
    #'xtick.labelsize': 12,
    #'ytick.labelsize': 12,
    'legend.fontsize': 12,
    'figure.dpi': 300 })

    template_half_max = np.max(template) / 2
    wf_half_max = np.max(wf_adcs[:150]) / 2
    template_idx = np.where(template >= template_half_max)[0][0]
    wf_idx = np.where(wf_adcs[:150] >= wf_half_max)[0][0]
    delta = template_idx - wf_idx

    # Alignement of the wf with template (and cut the final part if necessary)
    wf_adcs = np.roll(wf_adcs, delta)
    wf_adcs = wf_adcs[:len(template)]
    wf_adcs = wf_adcs[100:]
    template = template[100:]
    
    time = 16*np.arange(0, len(wf_adcs))
    new_time = np.linspace(time[0], time[-1], int(2.5 * len(time)))
    template_interp = interp1d(time, template, kind='linear')
    template_upsampled = template_interp(new_time)
    dt = (new_time[1] - new_time[0])

    params = wf.analyses[analysis_label].result['fit_params']
    area = wf.analyses[analysis_label].result['n_pe']
    e_area = wf.analyses[analysis_label].result['e_n_pe']
    r_squared = wf.analyses[analysis_label].result['r_squared']
    A_s, A_t, tau_s, tau_t, offset = wf.analyses[analysis_label].result['fit_params']
    eA_s, eA_t,  etau_s, etau_t,  eoffset = wf.analyses[analysis_label].result['fit_errors']

    if np.isnan(area):
        print("Area is NaN, skipping plot.")
        return

    lAr  = scintillation(new_time, A_s, A_t, tau_s, tau_t)
    conv = fft_convolution(new_time, *params, template_upsampled, dt)

    plt.figure(figsize=(6,4), constrained_layout=True)
    plt.plot(time, wf_adcs, 'blue', label = 'Signal')
    plt.plot(new_time, conv, 'k', label = 'Fit')

    info_text = (
        f"$A_s$ = {A_s:.2f} ± {eA_s:.2f}\n"
        f"$\\tau_s$ (ns) = {tau_s:.1f} ± {etau_s:.1f}\n"
        f"$A_t$ = {A_t:.3f} ± {eA_t:.3f}\n"
        f"$\\tau_t$ (ns) = {int(tau_t)} ± {int(etau_t)}\n"
        f"$N_{{PE}}$ = {int(area)} ± {int(e_area)}\n"
        f"R$^2$ = {r_squared:.3f}"
    )

    # Add the text box to the plot
    ax = plt.gca()
    box = AnchoredText(
        info_text, loc='upper right',
        frameon=True, bbox_transform=ax.transAxes, borderpad=0.5
    )
    ax.add_artist(box)

    plt.xlim(0, 16*len(time))
    plt.xlabel('Time(ns)')
    plt.ylabel('ADCs')
    plt.legend(loc = 'lower right')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.6)
    plt.show()

In [ ]:
check_convolution_result(grid_apa_FS.ch_wf_sets[104][10].waveforms[6], analysis_label)